<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_04_1_langchain_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 5: LangChain: Data Extraction**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 5 Material

* Part 5.1: Structured Output Parser [[Video]]() [[Notebook]](t81_559_class_05_1_langchain_data.ipynb)
* Part 5.2: Other Parsers (CSV, JSON, Pandas, Datetime)** [[Video]]() [[Notebook]](t81_559_class_05_2_parsers.ipynb)
* **Part 5.3: Pydantic Parser** [[Video]]() [[Notebook]](t81_559_class_05_3_pydantic.ipynb)
* Part 5.4: Custom Output Parser [[Video]]() [[Notebook]](t81_559_class_05_4_custom_parsers.ipynb)
* Part 5.5: Output-Fixing Parser [[Video]]() [[Notebook]](t81_559_class_05_5_output_fixing_parsers.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Att

# 5.3: Pydantic Parser

Pydantic is a data validation and settings management library in Python that uses Python type annotations. It's designed to allow for quick and easy data parsing and validation using Python's standard typing system.

Some of the key features of Pydantic:

* Data validation: It validates data to ensure it conforms to the expected format, converting types where necessary.
* Editor support: Pydantic models are classes that leverage Python's type hints, making them easy to use with modern editors that provide features like type checking and autocompletion.
* Error handling: It provides detailed and human-readable error reports to help identify where and why data failed to validate.
* Settings management: Pydantic is often used for managing settings/configurations, making it easy to load parameters from environment variables, JSON files, or other sources.
* Extensible: You can extend models with methods and properties, and use Pydantic's validation decorators to perform custom validation.
* Integration with other libraries: It works well with many other libraries, such as FastAPI for building APIs, enhancing their usability and functionality.

Overall, Pydantic is highly appreciated for its robustness and ease of use in ensuring that data inputs conform to specified formats, making it a valuable tool in modern Python development, especially in web development and data processing applications.

LangChain, a library designed to facilitate the building of applications with language models, offers various tools to manage and enhance interactions with these models. One of these tools is the PydanticOutputParser, which integrates Pydantic's powerful validation capabilities with the output of large language models (LLMs) like GPT.

The primary goal of the PydanticOutputParser is to ensure that the outputs from a language model are structured and adhere to a predefined schema. This is especially important in applications where consistent and reliable data formats are crucial, such as in data extraction tasks, API responses, or any scenario requiring subsequent automated processing of the model's output.

In [2]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

The following code uses an LLM to tell a joke. The pydantic parser ensures that the joke ends in a question mark. Such checking ensures that the somewhat random LLM produces output that aligns with our expectations.

In [3]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke about cats."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!')

In [4]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
    @validator('name')
    def validate_name(cls, value):
        parts = value.split()
        if len(parts) < 2:
            raise ValueError("Name must contain at least two words.")
        if not all(part[0].isupper() for part in parts):
            raise ValueError("Each word in the name must start with a capital letter.")
        return value

actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Cast Away', 'Saving Private Ryan', 'Toy Story', 'The Green Mile'])